# Part 2 

Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.

Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

Each year should be saved as a separate .csv.gz file

In [1]:
import pandas as pd
import numpy as np
import os, time, json
import tmdbsimple as tmdb 
with open('C:/Users/Sean/.edit/tmdb_api.json', 'r') as f:
    login = json.load(f)
from tqdm.notebook import tqdm_notebook
## Display the keys of the loaded dict
login.keys()

# Credential import
tmdb.API_KEY =  login['api-key']


In [2]:
#saving api call data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz',
 'title.akas.csv.gz',
 'title.ratings.csv.gz',
 '.ipynb_checkpoints',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2001.csv.gz']

In [3]:
def get_movie_certification(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    
    for c in releases['countries']:
        if c['iso_3166_1'] == "US":
            info['certifcation'] = c['certification']     
    return info

# function for json file creation

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [4]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')

basics


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...
143286,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
143287,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"
143288,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,84,Thriller
143289,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History"


In [5]:
#defining a list of years
YEARS_TO_GET = [2000,2001]

# defining an errors list
errors = [ ]

## Loop Extraction

In [10]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
#Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
# Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: create it
    if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
#Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
# saving movie ids to list
    movie_ids = df['tconst'].copy()    
# Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
 #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
 # Retrieve the data for the movie id
            temp = get_movie_certification(movie_id)  
 # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
# Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            #print(e)
            errors.append([movie_id, e])
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip",
                     index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/2650 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/2845 [00:00<?, ?it/s]

In [11]:
#any movie ids that caused an error?
print(f"- Total errors: {len(errors)}") 

- Total errors: 1113


Load in your csv.gz's of results for each year extracted.
Concatenate the data into 1 dataframe for the remainder of the analysis.
Once you have your data from the API, they would like you to perform some light EDA to show:
1. How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
* Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
2. How many movies are there in each of the certification categories (G/PG/PG-13/R)?
3. What is the average revenue per certification category?
4. What is the average budget per certification category?



In [13]:
#fetching data
movies_2k = pd.read_csv('Data/tmdb_api_results_2000.json', low_memory = False)
movies_2k1= pd.read_csv('Data/tmdb_api_results_2001.json', low_memory = False)

In [16]:
#concatinating into a single df
movies_2k_2k1 = pd.concat([movies_2k,movies_2k1])
movies_2k_2k1.head()

,"[{""imdb_id"": 0}","{""adult"": false","""backdrop_path"": ""/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg""","""belongs_to_collection"": null","""budget"": 10000000","""genres"": [{""id"": 35","""name"": ""Comedy""}","{""id"": 10402","""name"": ""Music""}","{""id"": 10749",...,"""poster_path"": ""/2RY46xorHLiLjeaCxft6e3n1idt.jpg""","""production_countries"": [].482","""release_date"": ""2001-01-01"".212","""revenue"": 0.2009","""runtime"": 0.234","""status"": ""Released"".2258","""tagline"": """".1637","""title"": ""Gay holocaust""","""video"": false.2252","""vote_count"": 0}]"


In [15]:
movies = pd.read_csv('Data/final_tmdb_data_2001.csv.gz', low_memory = False)
movies

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certifcation
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/ab5yL8zgRotrICzGbEl10z24N71.jpg,NaN,48000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'Italian', 'iso_639_1': 'it'...",Released,If they lived in the same century they'd be pe...,Kate & Leopold,0.0,6.32,1142.0,PG-13
2,tt0079644,0.0,/79axmuH1UGkB7m72jjB9rPff9om.jpg,NaN,0.0,"[{'id': 10752, 'name': 'War'}]",NaN,285529.0,id,November 1828,...,0.0,140.0,"[{'english_name': 'Indonesian', 'iso_639_1': '...",Released,NaN,November 1828,0.0,0.00,0.0,NaN
3,tt0089067,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,210258.0,es,El día de los albañiles 2,...,0.0,90.0,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,The laborers are back full of love and laughs.,El día de los albañiles 2,0.0,7.20,71.0,NaN
4,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.00,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,tt8795764,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,871624.0,en,New Breed,...,0.0,57.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,New Breed,0.0,0.00,0.0,NR
2256,tt8929248,0.0,NaN,NaN,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 18, '...",NaN,78417.0,ta,அழகான நாட்கள்,...,0.0,150.0,"[{'english_name': 'Tamil', 'iso_639_1': 'ta', ...",Released,NaN,Azhagana Naatkal,0.0,0.00,0.0,NaN
2257,tt9071078,0.0,NaN,NaN,0.0,"[{'id': 28, 'name': 'Action'}]",http://www.hkcinemagic.com/en/movie.asp?id=6627,201706.0,cn,致命密函,...,0.0,90.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,NaN,Chinese Heroes,0.0,3.00,2.0,NaN
2258,tt9099724,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,616033.0,ja,Rokushukan Private Moment,...,0.0,102.0,"[{'english_name': 'Japanese', 'iso_639_1': 'ja...",Released,NaN,Rokushukan Private Moment,0.0,0.00,0.0,NaN
